In [1]:
import hhgen.io as io
import hhgen.utils as utils
import matplotlib.pyplot as plt
import numpy as np
import skhep.math as skp

Welcome to ROOTaaS 6.06/05


In [2]:
files = ! ls /t3home/musella/Analysis/HH_8_0_28/src/HHbbgg_ETH/Flashgg/work/gen_analysis/*madgraph.root

In [3]:
reload(io)
def get_data(fname,proc):
    tdir = 'genDiphotonDumper/trees'

    untagged = tdir+'/'+proc+'_13TeV_NoTag_0'
    tagged = tdir+'/'+proc+'_13TeV_DoubleHTag_%d'

    return io.read_trees(fname,untagged,tagged,12) 

In [4]:
#hgg=H to 2 photons
#hbb=H to 2 b jets
#hh=H to 2 Higgs
reload(utils)
def calc_features(df):
    for part in "leadPho","subleadPho","leadJet","subleadJet":
        utils.calc_p4extra(df,part) #calculating pt, eta, phi, m for each 
    utils.calc_sump4(df,"hgg","leadPho","subleadPho") #calculating pt, eta, phi, m for sum of 2 photons
    utils.calc_sump4(df,"hbb","leadJet","subleadJet") #calculating pt, eta, phi, m for sum of 2 b jets
    utils.calc_sump4(df,"hh","hbb","hgg") #calculating pt, eta, phi, m for sum of 2 Higgs
    
    df["mtilda_x"]=df["hh_m"]-df["mgg"]-df["mbb"]+250.0 
    df["hgg_pt_hh_m"]=df["hgg_pt"]/df["hh_m"]
    df["hbb_pt_hh_m"]=df["hbb_pt"]/df["hh_m"]
    
    utils.calc_cos_theta_cs(df) #angle in Collins Sopper frame
    utils.calc_cos_theta(df,"hbb","leadJet") #helicity angles
    utils.calc_cos_theta(df,"hgg","leadPho")

In [5]:
import os

dfs = {}
for ifil in files:
    fname = os.path.basename(ifil) #output_GluGluToHHTo2B2G_node_10_13TeV-madgraph.root
    proc = fname.replace("output_","").replace(".root","").replace("-","_") #GluGluToHHTo2B2G_node_10_13TeV_madgraph
    df = get_data(ifil,proc)
    proc = proc.replace("GluGluToHHTo2B2G_","").replace("_13TeV_madgraph","")#node_10
    calc_features(df)
    dfs[proc] = df

In [6]:
#dfs["node_SM"].columns

In [1135]:
dfs["node_box"].hh_m.hist(range=[250,1000],bins=150)
plt.ylabel("Events")
plt.xlabel("hh$_m$ (in GeV/c$^2$)")
#plt.show()
plt.savefig("hh_m.png")

In [1138]:
plt.figure(2)
gbybdtcat=dfs["node_box"].groupby('bdtcat')
for icat in range(0,4):
    dfc=gbybdtcat.get_group(icat)
    dfc.hh_m.hist(range=[250,1000],bins=100,normed=True,histtype="step",label="bdtcat %d" % icat) 
plt.ylabel("Events")
plt.xlabel("hh$_m$ (in GeV/c$^2$)")
plt.legend()
#plt.show()
plt.savefig("hh_m_category_bdt.png")

In [1141]:
plt.figure(3)
gbymxcat=dfs["node_box"].groupby('mxcat')
for icat in range(0,5):
    dfc=gbymxcat.get_group(icat)
    dfc.hh_m.hist(range=[250,1000],bins=100,normed=True,histtype="step",label="mxcat %d" % icat) 
plt.ylabel("Events")
plt.xlabel("hh$_m$ (in GeV/c$^2$)")
plt.legend()
#plt.show()
plt.savefig("hh_m_category_mx.png")

In [1144]:
plt.figure(4)
dfs["node_box"].hh_pt.hist(range=[250,1000],bins=150)
plt.ylabel("Events")
plt.xlabel("hh$_{pT}$ (in GeV/c)")
#plt.show()
plt.savefig("hh_pt.png")

In [1147]:
plt.figure(5)
gbybdtcat=dfs["node_box"].groupby('bdtcat')
for icat in range(0,4):
    dfc=gbybdtcat.get_group(icat)
    dfc.hh_pt.hist(range=[250,1000],bins=100,normed=True,histtype="step",label="bdtcat %d" % icat) 
plt.ylabel("Events")
plt.xlabel("hh$_{pT}$ (in GeV/c)")
plt.legend()
#plt.show()
plt.savefig("hh_pt_category_bdt.png")

In [1150]:
plt.figure(6)
gbymxcat=dfs["node_box"].groupby('mxcat')
for icat in range(0,5):
    dfc=gbymxcat.get_group(icat)
    dfc.hh_pt.hist(range=[250,1000],bins=100,normed=True,histtype="step",label="mxcat %d" % icat) 
plt.ylabel("Events")
plt.xlabel("hh$_{pT}$ (in GeV/c)")
plt.legend()
#plt.show()
plt.savefig("hh_pt_category_mx.png")

In [1153]:
plt.figure(7)
dfs["node_box"].hh_eta.hist(range=[-5,5],bins=150)
plt.ylabel("Events")
plt.xlabel("$hh_{\eta}$")
#plt.show()
plt.savefig("hh_eta.png")

In [1156]:
plt.figure(8)
gbybdtcat=dfs["node_box"].groupby('bdtcat')
for icat in range(0,4):
    dfc=gbybdtcat.get_group(icat)
    dfc.hh_eta.hist(range=[-5,5],bins=100,normed=True,histtype="step",label="bdtcat %d" % icat) 
plt.ylabel("Events")
plt.xlabel("$hh_{\eta}$")
plt.legend()
#plt.show()
plt.savefig("hh_eta_category_bdt.png")

In [1159]:
plt.figure(9)
gbymxcat=dfs["node_box"].groupby('mxcat')
for icat in range(0,5):
    dfc=gbymxcat.get_group(icat)
    dfc.hh_eta.hist(range=[-5,5],bins=100,normed=True,histtype="step",label="mxcat %d" % icat) 
plt.ylabel("Events")
plt.xlabel("$hh_{\eta}$")
plt.legend()
#plt.show()
plt.savefig("hh_eta_category_mx.png")

In [1162]:
plt.figure(10)
dfs["node_box"].mtilda_x.hist(range=[250,1000],bins=150)
plt.ylabel("Events")
plt.xlabel("$\~{m}_X$ (in GeV/c$^2$)")
#plt.show()
plt.savefig("hh_mtilda_x.png")

In [1165]:
plt.figure(11)
gbybdtcat=dfs["node_box"].groupby('bdtcat')
for icat in range(0,4):
    dfc=gbybdtcat.get_group(icat)
    dfc.mtilda_x.hist(range=[250,1000],bins=100,normed=True,histtype="step",label="bdtcat %d" % icat) 
plt.ylabel("Events")
plt.xlabel("$\~{m}_X$ (in GeV/c$^2$)")
plt.legend()
#plt.show()
plt.savefig("hh_mtilda_x_category_bdt.png")

In [1168]:
plt.figure(12)
gbymxcat=dfs["node_box"].groupby('mxcat')
for icat in range(0,5):
    dfc=gbymxcat.get_group(icat)
    dfc.mtilda_x.hist(range=[250,1000],bins=100,normed=True,histtype="step",label="mxcat %d" % icat) 
plt.ylabel("Events")
plt.xlabel("$\~{m}_X$ (in GeV/c$^2$)")
plt.legend()
#plt.show()
plt.savefig("hh_mtilda_x_category_mx.png")

In [1171]:
plt.figure(13)
dfs["node_box"].hgg_pt_hh_m.hist(range=[0,1],bins=150)
plt.ylabel("Events")
plt.xlabel("pT($\gamma\gamma$)/M(jj$\gamma\gamma$) (in c)")
#plt.show()
plt.savefig("hgg_pt_hh_m.png")

In [1174]:
plt.figure(14)
gbybdtcat=dfs["node_box"].groupby('bdtcat')
for icat in range(0,4):
    dfc=gbybdtcat.get_group(icat)
    dfc.hgg_pt_hh_m.hist(range=[0,1],bins=100,normed=True,histtype="step",label="bdtcat %d" % icat) 
plt.ylabel("Events")
plt.xlabel("pT($\gamma\gamma$)/M(jj$\gamma\gamma$) (in c)")
plt.legend()
#plt.show()
plt.savefig("hgg_pt_hh_m_category_bdt.png")

In [1177]:
plt.figure(15)
gbymxcat=dfs["node_box"].groupby('mxcat')
for icat in range(0,5):
    dfc=gbymxcat.get_group(icat)
    dfc.hgg_pt_hh_m.hist(range=[0,1],bins=100,normed=True,histtype="step",label="mxcat %d" % icat) 
plt.ylabel("Events")
plt.xlabel("pT($\gamma\gamma$)/M(jj$\gamma\gamma$) (in c)")
plt.legend()
#plt.show()
plt.savefig("hgg_pt_hh_m_category_mx.png")

In [1180]:
plt.figure(16)
dfs["node_box"].hbb_pt_hh_m.hist(range=[0,1],bins=150)
plt.ylabel("Events")
plt.xlabel("pT(jj)/M(jj$\gamma\gamma$) (in c)")
#plt.show()
plt.savefig("hbb_pt_hh_m.png")

In [1183]:
plt.figure(17)
gbybdtcat=dfs["node_box"].groupby('bdtcat')
for icat in range(0,4):
    dfc=gbybdtcat.get_group(icat)
    dfc.hbb_pt_hh_m.hist(range=[0,1],bins=100,normed=True,histtype="step",label="bdtcat %d" % icat) 
plt.ylabel("Events")
plt.xlabel("pT(jj)/M(jj$\gamma\gamma$) (in c)")
plt.legend()
#plt.show()
plt.savefig("hbb_pt_hh_m_category_bdt.png")

In [1186]:
plt.figure(18)
gbymxcat=dfs["node_box"].groupby('mxcat')
for icat in range(0,5):
    dfc=gbymxcat.get_group(icat)
    dfc.hbb_pt_hh_m.hist(range=[0,1],bins=100,normed=True,histtype="step",label="mxcat %d" % icat) 
plt.ylabel("Events")
plt.xlabel("pT(jj)/M(jj$\gamma\gamma$) (in c)")
plt.legend()
#plt.show()
plt.savefig("hbb_pt_hh_m_category_mx.png")

In [1189]:
plt.figure(19)
np.abs(dfs["node_box"].cos_theta_cs).hist(bins=150)
plt.ylabel("Events")
plt.xlabel("|cos$\Theta_{CS}^*$|")
#plt.show()
plt.savefig("cos_theta_cs.png")

In [1192]:
plt.figure(20)
gbybdtcat=dfs["node_box"].groupby('bdtcat')
for icat in range(0,4):
    dfc=gbybdtcat.get_group(icat)
    np.abs(dfc.cos_theta_cs).hist(bins=100,normed=True,histtype="step",label="bdtcat %d" % icat) 
plt.ylabel("Events")
plt.xlabel("|cos$\Theta_{CS}^*$|")
plt.legend()
#plt.show()
plt.savefig("cos_theta_cs_category_bdt.png")

In [1195]:
plt.figure(21)
gbymxcat=dfs["node_box"].groupby('mxcat')
for icat in range(0,5):
    dfc=gbymxcat.get_group(icat)
    np.abs(dfc.cos_theta_cs).hist(bins=100,normed=True,histtype="step",label="mxcat %d" % icat) 
plt.ylabel("Events")
plt.xlabel("|cos$\Theta_{CS}^*$|")
plt.legend()
#plt.show()
plt.savefig("cos_theta_cs_category_mx.png")

In [1198]:
plt.figure(22)
np.abs(dfs["node_box"].cos_theta_hbb).hist(bins=150)
plt.ylabel("Events")
plt.xlabel("|cos$\Theta_{bb}^*$|")
#plt.show()
plt.savefig("cos_theta_bb.png")

In [1201]:
plt.figure(23)
gbybdtcat=dfs["node_box"].groupby('bdtcat')
for icat in range(0,4):
    dfc=gbybdtcat.get_group(icat)
    np.abs(dfc.cos_theta_hbb).hist(bins=100,normed=True,histtype="step",label="bdtcat %d" % icat) 
plt.ylabel("Events")
plt.xlabel("|cos$\Theta_{bb}^*$|")
plt.legend()
#plt.show()
plt.savefig("cos_theta_bb_category_bdt.png")

In [1204]:
plt.figure(24)
gbymxcat=dfs["node_box"].groupby('mxcat')
for icat in range(0,5):
    dfc=gbymxcat.get_group(icat)
    np.abs(dfc.cos_theta_hbb).hist(bins=100,normed=True,histtype="step",label="mxcat %d" % icat) 
plt.ylabel("Events")
plt.xlabel("|cos$\Theta_{bb}^*$|")
plt.legend()
#plt.show()
plt.savefig("cos_theta_bb_category_mx.png")

In [1207]:
plt.figure(25)
np.abs(dfs["node_box"].cos_theta_hgg).hist(bins=150)
plt.ylabel("Events")
plt.xlabel("|cos$\Theta_{\gamma\gamma}^*$|")
#plt.show()
plt.savefig("cos_theta_gg.png")

In [1210]:
plt.figure(26)
gbybdtcat=dfs["node_box"].groupby('bdtcat')
for icat in range(0,4):
    dfc=gbybdtcat.get_group(icat)
    np.abs(dfc.cos_theta_hgg).hist(bins=100,normed=True,histtype="step",label="bdtcat %d" % icat) 
plt.ylabel("Events")
plt.xlabel("|cos$\Theta_{\gamma\gamma}^*$|")
plt.legend()
#plt.show()
plt.savefig("cos_theta_gg_category_bdt.png")

In [1213]:
plt.figure(27)
gbymxcat=dfs["node_box"].groupby('mxcat')
for icat in range(0,5):
    dfc=gbymxcat.get_group(icat)
    np.abs(dfc.cos_theta_hgg).hist(bins=100,normed=True,histtype="step",label="mxcat %d" % icat) 
plt.ylabel("Events")
plt.xlabel("|cos$\Theta_{\gamma\gamma}^*$|")
plt.legend()
#plt.show()
plt.savefig("cos_theta_gg_category_mx.png")

In [280]:
#hdf: hierarchial data format

for proc,df in dfs.items():
    df.to_hdf(proc+".hd5",proc,mode="w") #mode w: write

In [281]:
! ls *.hd5

node_10.hd5  node_13.hd5  node_4.hd5  node_7.hd5  node_box.hd5
node_11.hd5  node_2.hd5   node_5.hd5  node_8.hd5  node_SM.hd5
node_12.hd5  node_3.hd5   node_6.hd5  node_9.hd5
